# Raster Mask

Created 2019-01-15
This notebook is used to create connected density thresholds in a population raster. It applies a mask to isolate urban cores (>1500 ppl/km2) and then a second mask to capture pixels w/ 300 ppl per km2 that are connected to the urban cores. See: Florczyk A J 2019 Description of the GHS Urban Centre Database 2015; Public Release 2019 Version 1.0 KJ-02-19-103-EN-N

This step needs to be repeated for each raster dataset for which you want to isolate urban populations.

Note: Check crs of Raster 'EPSG:4326' ... or EPSG: 54009

### Dependencies

In [ ]:
import rasterio
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import matplotlib.pyplot as plt

### Inputs

In [ ]:
# File paths

IN = ''
OUT = ''

### Functions

Note: These have been moved to a .py file in the src directory

In [ ]:
def raster_mask_poly(raster, band, density, crs):
    """Fucntion makes numpy array for of raster then applies mask.
    requires rasterio object, band number, and density as float on int and crs of raster
    returns dict of polygons and numpy mask
    
    Args: raster = input raster as rasterio object
          band = band of raster
          density = density to mask
          crs = crs of raster
    """

    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros 
    mask[mask >= density] = 1 
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326) removed 2019-01-15
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return mask, geom, poly_gdf

In [ ]:
def raster_mask(raster, band, density):
    """Function returns raster mask as numpy array
    
    Args: raster = input raster as rasterio object
          band = band of raster
          density = density to mask
    """
    
    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros and ones
    mask[mask >= density] = 1
    
    return mask

In [ ]:
def raster_poly(raster, band, crs):
    """Fucntion makes polygons for of raster returns dict of polygons
    
        Args: raster = input raster as rasterio object
              band = band of raster
              crs = crs
    """

    mask = raster.read(band)
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326).
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return poly_gdf

In [ ]:
def raster_gpd(array, raster, crs):
    """ Function takes an np array, raster
    and returns polygons from the np array in the raster CRS
    
    Args: array = array to make polygons
          raster = raster to use as template
          crs = crs
    """
    for geom, val in rasterio.features.shapes(array, transform = raster.transform):

        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326). Removed # WGS84 (EPSG 4326) 
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) 

    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)
    
    return poly_gdf

In [ ]:
def raster_write(meta, array, file_out):
    """ Function to write out a raster file with an np array
    requires meta data for raster, np array & file out path and name
    
    Args: meta = meta data
          array = data to write into a tif
          file_out = file name
    """
    
    kwargs = meta

    # Update kwargs (change in data type)
    kwargs.update(dtype=rasterio.float32, count = 1)

    with rasterio.open(file_out, 'w', **kwargs) as dst:
        dst.write_band(1, array.astype(rasterio.float32))

### Connected Pixels 1500 - 300

In [ ]:
# Get native functions from SRC 
import os

os.getcwd()
os.chdir('') # set directory path
os.getcwd()

from src import raster_funcs

In [ ]:
# Open Raster and Set CRC 

file = 'LS15_w001001_Clip'
raster = rasterio.open('data/interim/'+file+'.tif')
crs = raster.crs

In [ ]:
# Create Masks

mask300 = raster_mask(raster, 1, 300)
mask1500 = raster_mask(raster, 1, 1500)

In [ ]:
# Find connected pixels

mask1500c300 = raster_funcs.remove_isolated_pixels(mask1500, mask300)

In [ ]:
# Turn into geo pandas 

poly_gdf_1500c300 = raster_gpd(mask1500c300, raster, crs)

In [ ]:
# Check data and check Length

print((poly_gdf_1500c300.head(6)))
print(len(poly_gdf_1500c300))

In [ ]:
# drop first row, which is bounding box of Africa
poly_gdf_1500c300_drop = poly_gdf_1500c300.drop(poly_gdf_1500c300.index[0], axis = 0)
print((poly_gdf_1500c300_drop.head(6)))
print(len(poly_gdf_1500c300_drop))

In [ ]:
# Write as shape file

poly_gdf_1500c300_drop.to_file(downloads+file+'_1500c300.shp', driver='ESRI Shapefile')